# **Examples to use on Oracle Data Science!**

# **1.1 Create Custom Conda**

Open a command prompt and use:

- odsc conda create -n <NAME_OF_CONDA> -s <NAME_OF_CONDA> -v 1.0

Then, activate the Conda Environment
- conda activate /home/datascience/conda/<NAME_OF_CONDA>

Then, take a note of the yolo conda location
- conda env list

Go to the <NAME_OF_CONDA> folder
- cd /home/datascience/conda/<NAME_OF_CONDA>

# **1.2 Publish Custom Conda to Object Storage**

In [ ]:
#1.st define bucket and namespace where to store
!odsc conda init -b <BUCKET_NAME> -n <NAMESPACE> -a resource_principal

In [ ]:
#2.nd publish logo_object_detection_conda conda env to bucket
!odsc conda publish -s <NAME_OF_CONDA>

# **2 Authenticate Against OCI**

# First. Generate a Config file and upload to OCI, API Keys

Lin: https://docs.oracle.com/en-us/iaas/Content/API/Concepts/apisigningkey.htm#apisigningkey_topic_How_to_Generate_an_API_Signing_Key_Console

In [ ]:
# Check Authentication with OCI
def oci_authenticate(debug):
    
    # Set up and read in config file to authenticate against OCI
    config = oci.config.from_file('~/.oci/config', 'DEFAULT')       #config file

    # Create a Service Client to Identity API
    identity = oci.identity.IdentityClient(config)

    # Check if successful response received
    if identity.get_user(config['user']).status == 200:
        
        # Get the current User Information from API
        user = identity.get_user(config['user']).data

        if debug == 'y':
            print('\nAuthentication Successful!')
            print('\nOCI User Logged In:', user.name)
            print('OCI User Email:', user.email)
    else:
        print('Authentication Failed!')
        print('HTTP Request Error Code:', identity.get_user(config['user']).status)

    # Return config object to be used to initialise AI Language Client
    return config

# Authenticate against OCI
config = oci_authenticate('y')

# **3 Autonomous Database Connection + Push results to table in ADW**

In [ ]:
#First download ADW wallet and store in /home/datascience/<PATH_TO_WALLET>

########################################################
# Enter connection information and wallet file directory
database_name = '<DATABASE_NAME>'
database_user = '<USER>'
database_password = '******************************************'
wallet_storage_directory = os.path.join(os.path.expanduser("~"),"<PATH_TO_WALLET")    

#ads.set_documentation_mode(False)

# Create the wallet directory if missing: 
os.makedirs(wallet_storage_directory, mode=0o700, exist_ok=True)

wallet_path = os.path.join(wallet_storage_directory, database_name)

# Prepare to store ADB connection information
adb_config = os.path.join(wallet_storage_directory, '.credentials')

# Write a configuration file for login creds.
config = configparser.ConfigParser()
config.read(adb_config)
config[database_name] = {'tns_admin': wallet_path,
                         'sid': '{}_medium'.format(database_name.lower()),
                         'user': database_user,
                         'password': database_password}
with open(adb_config, 'w') as configfile:
    config.write(configfile)
    
# Read in the credentials configuration files
my_config = configparser.ConfigParser()
my_config.read(adb_config)

# Access a setting
print(my_config[database_name].get('user'))

# Limit the information to a specific database
my_creds = my_config[database_name]
print(my_creds.get('user'))


# extract the wallet
wallet_file = 'Wallet_{}.zip'.format(database_name)
wallet_filename = os.path.join(wallet_storage_directory, wallet_file)
if not os.path.exists(wallet_filename):
    print("The file {} does not exist.".format(wallet_filename))
    print("Please copy the Wallet file, {}, into the directory {} then rerun this cell.".format(wallet_file, wallet_filename))
else:
    os.makedirs(wallet_path, mode=0o700, exist_ok=True)
    with ZipFile(wallet_filename, 'r') as zipObj:
        zipObj.extractall(wallet_path)

# Update the sqlnet.ora

sqlnet_path = os.path.join(wallet_path, 'sqlnet.ora')
sqlnet_original_path = os.path.join(wallet_path, 'sqlnet.ora.original')
sqlnet_backup_path = os.path.join(wallet_path, 'sqlnet.ora.backup')
if not os.path.exists(sqlnet_original_path):
    shutil.copy(sqlnet_path, sqlnet_original_path)
if os.path.exists(sqlnet_path):
    shutil.copy(sqlnet_path, sqlnet_backup_path)
sqlnet_re = re.compile('(WALLET_LOCATION\s*=.*METHOD_DATA\s*=.*DIRECTORY\s*=\s*\")(.*)(\".*)', 
                       re.IGNORECASE)
tmp = NamedTemporaryFile()
with open(sqlnet_path, 'rt') as sqlnet:
    for line in sqlnet:
        tmp.write(bytearray(sqlnet_re.subn(r'\1{}\3'.format(wallet_path), line)[0], 
                            encoding='utf-8'))
tmp.flush()
shutil.copy(tmp.name, sqlnet_path)
tmp.close()

# Add TNS_ADMIN to the environment
os.environ['TNS_ADMIN'] = config[database_name].get('tns_admin')

# Test the database connection
creds = config[database_name]
connect = 'sqlplus ' + creds.get('user') + '/' + creds.get('password') + '@' + creds.get('sid')
print(os.popen(connect).read())

In [ ]:
# Get the URI to connect to the database
uri='oracle+cx_oracle://' + creds.get('user') + ':' + creds.get('password') + '@' + creds.get('sid')

In [ ]:
#read csv as pandas df
pd_to_adw = pd.read_csv("/home/datascience/xxxx.csv")

#push df to ADW
pd_to_adw.to_sql('<NEW_TABLE_NAME>', con=uri, index=False, if_exists="append")\

----

# **3. Create a Data Science Jobs**

In [ ]:
import ads
from ads.common.oci_logging import OCILogGroup, OCILog
from ads.jobs import Job, infrastructure, PythonRuntime, GitPythonRuntime, NotebookRuntime, ScriptRuntime, DataScienceJob
from datetime import datetime, timedelta
from ads import set_auth

set_auth(auth='resource_principal') #uses the notebook session as authentication

In [ ]:
# Define an OCI Data Science job to run a python script
## add all OCIDS for each services
job = (
    Job(name="<NEW JOB NAME>")
    .with_infrastructure(
        DataScienceJob()
        .with_compartment_id("ocid1.compartment.oc1..x")
        .with_project_id("ocid1.datascienceproject.oc1.x")
        .with_subnet_id("ocid1.subnet.oc1.x")
        .with_shape_name("VM.Standard2.1")
        .with_block_storage_size(50)
        .with_log_group_id("ocid1.loggroup.oc1.x")  
        .with_log_id("ocid1.log.oc1.xa")
    )
    .with_runtime(
        ScriptRuntime()
        .with_source("/home/datascience/<DIRECTORY OF .PY FILE TO USE IN JOB>")   ## entire script to run
        .with_custom_conda("oci://<BUCKET>@<NAMESPACE>/<DIRECTORY OF CUSTOM CONDA>") #refer to where custom conda is stored
        .with_environment_variable(ENV_INPUT"oci://<BUCKET>@<NAMESPACE>/input/",
                                   ENV_OUTPUT="oci://<BUCKET>@<NAMESPACE>/output/")  
    )
    .create()       
)

In [ ]:
job_2 = job.run()

In [ ]:
job_run_ = job_2.watch()